In [2]:
from pathlib import Path
import duckdb
import pandas as pd
db_path = str(Path.cwd().parent / 'hdbc.duckdb')

conn = duckdb.connect(db_path)


# d_ovw = conn.execute("SELECT * FROM dim_overview").df()

In [12]:
guest_month = conn.execute("""
            SELECT 
                month,	
                hotel,	
                guests
            FROM avg_guest_month
        """)
guest_month_df = guest_month.df()

In [1]:
pip install --upgrade altair streamlit

  Using cached altair-5.3.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached toolz-0.12.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached tenacity-8.3.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached altair-5.3.0-py3-none-any.whl (857 kB)
   ---------------------------------------- 0.0/8.6 MB ? eta -:--:--
   ------------------------------------

In [13]:
guest_month_df

,month,hotel,guests
0,October,Resort hotel,2396.500000
1,December,Resort hotel,1892.500000
2,March,Resort hotel,2305.500000
3,September,Resort hotel,2086.500000
4,February,Resort hotel,2153.000000
5,August,Resort hotel,2506.000000
6,June,Resort hotel,2062.000000
7,April,Resort hotel,2343.500000
8,January,Resort hotel,1592.000000
9,May,Resort hotel,2431.500000


In [2]:
cancel_month = conn.execute("""
        SELECT 
            deposit_type,
            AVG(CASE WHEN is_canceled = 1 THEN 100.0 ELSE 0 END) AS cancellation_percentage
        FROM dim_hotel
        GROUP BY deposit_type
    """)

cancel_month_df = cancel_month.df()

In [3]:
cancel_month_df

,deposit_type,cancellation_percentage
0,No Deposit,28.382164
1,Non Refund,99.362446
2,Refundable,22.222222


In [3]:
conn.close()

In [43]:
cancel_month = conn.execute("""
            SELECT 
                arrival_date_month,
                AVG(CASE WHEN is_canceled = 1 THEN 100.0 ELSE 0 END) AS cancellation_percentage,
                hotel
            FROM dim_hotel
            GROUP BY arrival_date_month, hotel
            ORDER BY arrival_date_month
        """)


cancel_month_results = cancel_month.fetchall()


cancel_month_df = pd.DataFrame(cancel_month_results, columns=['arrival_date_month', 'cancellation_percentage', 'hotel'])



In [44]:

cancel_month_df

,arrival_date_month,cancellation_percentage,hotel
0,April,46.323529,City Hotel
1,April,29.343308,Resort Hotel
2,August,40.097963,City Hotel
3,August,33.449121,Resort Hotel
4,December,23.829305,Resort Hotel
5,December,42.110358,City Hotel
6,February,25.620367,Resort Hotel
7,February,38.288016,City Hotel
8,January,39.668094,City Hotel
9,January,14.819881,Resort Hotel


In [39]:
cancel_rate = conn.execute("""
            SELECT
    'Resort hotel' AS hotel,
    (stays_in_weekend_nights + stays_in_week_nights) AS total_nights,
    COUNT(*) / (SELECT COUNT(*) FROM resort_hotel) * 100 AS rel_num_bookings
FROM resort_hotel
GROUP BY total_nights

UNION ALL

SELECT
    'City hotel' AS hotel,
    (stays_in_weekend_nights + stays_in_week_nights) AS total_nights,
    COUNT(*) / (SELECT COUNT(*) FROM city_hotel) * 100 AS rel_num_bookings
FROM city_hotel
GROUP BY total_nights;

                    """)
cancel_rate_df = cancel_rate.df()

CatalogException: Catalog Error: Table with name resort_hotel does not exist!
Did you mean "dim_hotel"?
LINE 6: FROM resort_hotel
             ^

In [35]:

cancel_rate_df

,hotel,total_days_stayed,percentage_of_total_days
0,Resort Hotel,173001.0,42.271971
1,City Hotel,236256.0,57.728029


In [25]:
cancel_rate_df

In [34]:
from pathlib import Path
import duckdb
db_path = str(Path.cwd().parent / 'hdbc.duckdb')
conn = duckdb.connect(db_path)
d_ovw = conn.sql("SELECT * FROM dim_overview").df()

IOException: IO Error: File is already open in 
C:\Users\njhus\AppData\Local\Programs\Python\Python312\python.exe (PID 8304)

In [45]:
conn.close()

In [23]:
total_count = conn.sql("SELECT COUNT(*) FROM dim_overview").fetchall()[0][0]
query_result = conn.sql("SELECT COUNT(CASE WHEN hotel='Resort Hotel' THEN 1 END) AS resort_count, COUNT(CASE WHEN hotel='City Hotel' THEN 1 END) AS city_count FROM dim_overview").fetchall()[0]

resort_count, city_count = query_result
resort_percentage = (resort_count / total_count) * 100
city_percentage = (city_count / total_count) * 100

print(f"Resort Hotel Percentage: {resort_percentage:.2f}%")
print(f"City Hotel Percentage: {city_percentage:.2f}%")


Resort Hotel Percentage: 33.55%
City Hotel Percentage: 66.45%


In [26]:
# Execute a single SQL query to get the counts for 'Resort Hotel' and 'City Hotel' in one go
query_result = conn.sql("""
    SELECT 
        COUNT(CASE WHEN hotel='Resort Hotel' THEN 1 END) AS resort_count,
        COUNT(CASE WHEN hotel='City Hotel' THEN 1 END) AS city_count,
        COUNT(*) AS total_count
    FROM dim_overview
""").fetchall()

# Unpack the values from the query result
# resort_count, city_count, total_count = query_result

# Calculate the percentages directly without needing separate queries
resort_percentage = (resort_count / total_count) * 100
city_percentage = (city_count / total_count) * 100

# Print the results
print(f"Resort Hotel Percentage: {resort_percentage:.2f}%")
print(f"City Hotel Percentage: {city_percentage:.2f}%")


Resort Hotel Percentage: 33.55%
City Hotel Percentage: 66.45%


In [43]:
query_result = conn.sql("""
SELECT 
    COUNT(CASE WHEN hotel='Resort Hotel' THEN 1 END) AS resort_count,
    COUNT(CASE WHEN hotel='City Hotel' THEN 1 END) AS city_count,
    COUNT(*) AS total_count
FROM dim_overview
""").fetchall()

# Extracting counts from the query result
resort_count, city_count, total_count = query_result[0]

# Calculating percentages
resort_percentage = (resort_count / total_count) * 100
city_percentage = (city_count / total_count) * 100

# Pie chart
labels = ['Resort Hotel', 'City Hotel']
sizes = [resort_count, city_count]
explode = (0.1, 0)

fig, ax = plt.subplots(figsize=(8, 6))
ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=140)
ax.axis('equal')
ax.set_title('Hotel Type Distribution', fontsize=12)

Text(0.5, 1.0, 'Hotel Type Distribution')

In [42]:
import streamlit as st
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
from pathlib import Path
import io
import base64

In [45]:
resort_count, city_count, total_count 

(40060, 79330, 119390)

In [59]:
cancel_rate = conn.sql("""
    SELECT
        is_cancelled as Cancellation,
        (COUNT(CASE WHEN is_canceled='1' THEN 1 END) / COUNT(*)) * 100 AS cancelled,
        (COUNT(CASE WHEN is_canceled='0' THEN 1 END) / COUNT(*)) * 100 AS not_cancelled
    FROM dim_overview   
    """).df()

In [54]:
cancel_rate

,cancelled,not_cancelled
0,37.041628,62.958372


In [67]:

cancel_rate = conn.execute("""
    SELECT 
        (COUNT(CASE WHEN is_canceled='1' THEN 1 END) / COUNT(*)) * 100 AS cancelled,
        (COUNT(CASE WHEN is_canceled='0' THEN 1 END) / COUNT(*)) * 100 AS not_cancelled
    FROM dim_overview   
""")

cancel_rate_df = cancel_rate.df()

In [68]:
cancel_rate_df

,cancelled,not_cancelled
0,37.041628,62.958372


In [62]:
cancel_rate = conn.execute("""
        SELECT 
            is_canceled as Cancellation,
            (COUNT(CASE WHEN is_canceled='1' THEN 1 END) / COUNT(*)) * 100 AS cancelled,
            (COUNT(CASE WHEN is_canceled='0' THEN 1 END) / COUNT(*)) * 100 AS not_cancelled
        FROM dim_overview   
        GROUP BY Cancellation
    """)


cancel_rate_df = cancel_rate.df()

In [63]:
cancel_rate_df

,Cancellation,cancelled,not_cancelled
0,0,0.0,100.0
1,1,100.0,0.0


In [69]:
cancel_rate = conn.execute("""
    SELECT 
        is_canceled as Cancellation,
        (COUNT(CASE WHEN is_canceled='1' THEN 1 END) / COUNT(*)) * 100 AS cancelled,
        (COUNT(CASE WHEN is_canceled='0' THEN 1 END) / COUNT(*)) * 100 AS not_cancelled
    FROM dim_overview   
    GROUP BY is_canceled
""")

cancel_rate_df = cancel_rate.df()

In [70]:
cancel_rate_df

,Cancellation,cancelled,not_cancelled
0,0,0.0,100.0
1,1,100.0,0.0


In [71]:
cancel_rate = conn.execute("""
    SELECT 
        'Cancellation Rate' as Cancellation,
        (COUNT(CASE WHEN is_canceled='1' THEN 1 END) / COUNT(*)) * 100 AS cancelled,
        (COUNT(CASE WHEN is_canceled='0' THEN 1 END) / COUNT(*)) * 100 AS not_cancelled
    FROM dim_overview
""")

cancel_rate_df = cancel_rate.df()


In [72]:
cancel_rate_df

,Cancellation,cancelled,not_cancelled
0,Cancellation Rate,37.041628,62.958372


In [17]:
conn.close()

In [75]:
cancel_rate = conn.execute("""
    SELECT 
        CASE WHEN is_canceled='1' THEN 'Cancelled' ELSE 'Not Cancelled' END AS Cancellation,
        (COUNT(*) / (SELECT COUNT(*) FROM dim_overview)) * 100 AS Percentage
    FROM dim_overview
    GROUP BY is_canceled
""")

cancel_rate_df = cancel_rate.df()

In [76]:
cancel_rate_df

,Cancellation,Percentage
0,Not Cancelled,62.958372
1,Cancelled,37.041628


In [84]:
Market_Segment = conn.execute("""
    SELECT market_segment as "Market Segment", COUNT(*) as count
    FROM dim_overview
    GROUP BY market_segment
    ORDER BY count DESC
    """)

Market_Segment_df = Market_Segment.df()

    # st.subheader("Market Segment")
    # st.bar_chart(
    # Market_Segment_df,
    # x="Market Segment",
    # y="count",
    # color=["#859887"],
    # height=500)

In [86]:
Market_Segment_df

,Market Segment,count
0,Online TA,56477
1,Offline TA/TO,24219
2,Groups,19811
3,Direct,12606
4,Corporate,5295
5,Complementary,743
6,Aviation,237
7,Undefined,2


In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [13]:
guest_month = conn.execute("""
            SELECT 
                arrival_date_month,
                SUM(adults + children + babies) AS sum_number_of_guest,
                hotel
            FROM dim_hotel
            GROUP BY arrival_date_month, hotel
            ORDER BY arrival_date_month
        """)

guest_month_df = guest_month.df()

In [14]:
guest_month_df

,arrival_date_month,sum_number_of_guest,hotel
0,April,15101.0,City Hotel
1,April,6899.0,Resort Hotel
2,August,19401.0,City Hotel
3,August,11370.0,Resort Hotel
4,December,5051.0,Resort Hotel
5,December,8145.0,City Hotel
6,February,5923.0,Resort Hotel
7,February,9376.0,City Hotel
8,January,6721.0,City Hotel
9,January,3803.0,Resort Hotel


In [15]:
guest_month_df = pd.DataFrame(guest_month.fetchall(), columns=['arrival_date_month', 'sum_number_of_guest', 'hotel'])

        # Adjusting the sum of the number of guests for July and August
guest_month_df.loc[(guest_month_df["arrival_date_month"] == "July") | (guest_month_df["arrival_date_month"] == "August"),
                        "sum_number_of_guest"] /= 3
guest_month_df.loc[~((guest_month_df["arrival_date_month"] == "July") | (guest_month_df["arrival_date_month"] == "August")),
                        "sum_number_of_guest"] /= 2

month_order = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
guest_month_df['arrival_date_month'] = pd.Categorical(guest_month_df['arrival_date_month'], categories=month_order, ordered=True)
guest_month_df = guest_month_df.sort_values(by='arrival_date_month')


In [16]:
guest_month_df

,arrival_date_month,sum_number_of_guest,hotel
